# Lesson 7 - Starter Code

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics

%matplotlib inline
import matplotlib.pyplot as plt

## Create sample data and fit a model

In [2]:
np.random.seed(seed = 100)
df = pd.DataFrame({"x": range(100), "y": range(100)})
biased_df  = df.copy()
biased_df.loc[:20, "x"] = 1
biased_df.loc[:20, "y"] = 1

def append_jitter(series):
    jitter = np.random.random_sample(size=100)
    return series + jitter

df["x"] = append_jitter(df.x)
df["y"] = append_jitter(df.y)

biased_df["x"] = append_jitter(biased_df.x)
biased_df["y"] = append_jitter(biased_df.y)

In [3]:
df.head()

,x,y
0,0.543405,0.778289
1,1.278369,1.779598
2,2.424518,2.610328
3,3.844776,3.309000
4,4.004719,4.697735


In [4]:
biased_df.head()

,x,y
0,1.417091,1.009934
1,1.695591,1.789766
2,1.424847,1.275313
3,1.858114,1.717740
4,1.846932,1.421356


In [5]:
# fit
lm = linear_model.LinearRegression().fit(df[["x"]], df["y"])
print(metrics.mean_squared_error(df["y"], lm.predict(df[["x"]])))

0.175065131142


In [6]:
# biased fit
lm = linear_model.LinearRegression().fit(biased_df[["x"]], biased_df["y"])
print(metrics.mean_squared_error(df["y"], lm.predict(df[["x"]])))

0.187115987374


## Cross validation
### Introduction to cross validation with bike share data from last time. We will be modeling casual ridership.

In [7]:
from sklearn import cross_validation

np.random.seed(seed = 100)
wd = "../../../../data/"
bikeshare = pd.read_csv(wd + "bikeshare.csv")

/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
bikeshare.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Create dummy variables and set outcome (dependent) variable

In [8]:
weather = pd.get_dummies(bikeshare.weathersit, prefix = "weather")
modeldata = bikeshare[["temp", "hum"]].join(weather[["weather_1", "weather_2", "weather_3"]])
y = bikeshare.casual 

In [15]:
modeldata.head()

,temp,hum,weather_1,weather_2,weather_3
0,0.24,0.81,1,0,0
1,0.22,0.80,1,0,0
2,0.22,0.80,1,0,0
3,0.24,0.75,1,0,0
4,0.24,0.75,1,0,0


### Create a cross valiation with 5 folds

In [17]:
kf = cross_validation.KFold(len(modeldata), n_folds = 22, shuffle = True)

In [18]:
mse_values = []
scores = []
n = 0
print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf:
    lm = linear_model.LinearRegression().fit(modeldata.iloc[train_index], y.iloc[train_index])
    mse_values.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(modeldata.iloc[test_index])))
    scores.append(lm.score(modeldata, y))
    n+=1
    print("Model: %d" % n)
    print("MSE: %f" % mse_values[n - 1])
    print("R2 : %f" % scores[n - 1])

print("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print("Mean of MSE for all folds: %f" % np.mean(mse_values))
print("Mean of R2 for all folds : %f" % np.mean(scores))

~~~~ CROSS VALIDATION each fold ~~~~
Model: 1
MSE: 1807.382278
R2 : 0.311931
Model: 2
MSE: 1549.234115
R2 : 0.311932
Model: 3
MSE: 1378.815461
R2 : 0.311920
Model: 4
MSE: 1629.574030
R2 : 0.311930
Model: 5
MSE: 1353.577535
R2 : 0.311924
Model: 6
MSE: 1650.126041
R2 : 0.311933
Model: 7
MSE: 1287.419388
R2 : 0.311923
Model: 8
MSE: 1693.022173
R2 : 0.311928
Model: 9
MSE: 1829.658937
R2 : 0.311930
Model: 10
MSE: 1622.474520
R2 : 0.311930
Model: 11
MSE: 1877.394627
R2 : 0.311925
Model: 12
MSE: 1521.349821
R2 : 0.311930
Model: 13
MSE: 1729.295396
R2 : 0.311928
Model: 14
MSE: 1916.201363
R2 : 0.311929
Model: 15
MSE: 1576.969455
R2 : 0.311931
Model: 16
MSE: 1620.335259
R2 : 0.311929
Model: 17
MSE: 1635.837736
R2 : 0.311927
Model: 18
MSE: 1587.101108
R2 : 0.311932
Model: 19
MSE: 1857.224145
R2 : 0.311927
Model: 20
MSE: 1859.523736
R2 : 0.311922
Model: 21
MSE: 2094.949542
R2 : 0.311930
Model: 22
MSE: 1733.630280
R2 : 0.311932
~~~~ SUMMARY OF CROSS VALIDATION ~~~~
Mean of MSE for all folds: 1673.

In [19]:
lm = linear_model.LinearRegression().fit(modeldata, y)
print("~~~~ Single Model ~~~~")
print("MSE: %f" % metrics.mean_squared_error(y, lm.predict(modeldata)))
print("R2 : %f" % lm.score(modeldata, y))

~~~~ Single Model ~~~~
MSE: 1672.581108
R2 : 0.311935


### Check
While the cross validated approach here generated more overall error, which of the two approaches would predict new data more accurately: the single model or the cross validated, averaged one? Why?

Answer: ? The single model, the 

### Advanced: There are ways to improve our model with regularization.
Let's check out the effects on $MSE$ and $R^2$

In [16]:
lm = linear_model.LinearRegression().fit(modeldata, y)
print("~~~ OLS ~~~")
print("OLS MSE: %f" % metrics.mean_squared_error(y, lm.predict(modeldata)))
print("OLS R2 : %f" % lm.score(modeldata, y))

lm = linear_model.Lasso().fit(modeldata, y)
print("~~~ Lasso ~~~")
print("Lasso MSE: %f" % metrics.mean_squared_error(y, lm.predict(modeldata)))
print("Lasso R2 : %f" % lm.score(modeldata, y))

lm = linear_model.Ridge().fit(modeldata, y)
print("~~~ Ridge ~~~")
print("Ridge MSE: %f" % metrics.mean_squared_error(y, lm.predict(modeldata)))
print("Ridge R2 : %f" % lm.score(modeldata, y))

~~~ OLS ~~~
OLS MSE: 1672.581108
OLS R2 : 0.311935
~~~ Lasso ~~~
Lasso MSE: 1725.415816
Lasso R2 : 0.290199
~~~ Ridge ~~~
Ridge MSE: 1672.604901
Ridge R2 : 0.311925


### Figuring out the alphas can be done by "hand"

In [20]:
alphas = np.logspace(-10, 10, 21)
for a in alphas:
    print("Alpha: %f" % a)
    lm = linear_model.Ridge(alpha = a)
    lm.fit(modeldata, y)
    print(lm.coef_)
    print(metrics.mean_squared_error(y, lm.predict(modeldata)))

Alpha: 0.000000
[ 112.68901765  -84.01121684  -24.68489063  -21.00314493  -21.71893628]
1672.58110765
Alpha: 0.000000
[ 112.68901765  -84.01121684  -24.68489061  -21.00314491  -21.71893626]
1672.58110765
Alpha: 0.000000
[ 112.68901765  -84.01121684  -24.6848904   -21.00314471  -21.71893606]
1672.58110765
Alpha: 0.000000
[ 112.68901763  -84.01121682  -24.68488837  -21.00314268  -21.71893403]
1672.58110765
Alpha: 0.000001
[ 112.68901745  -84.01121667  -24.68486804  -21.00312237  -21.71891373]
1672.58110765
Alpha: 0.000010
[ 112.68901562  -84.01121509  -24.68466472  -21.00291929  -21.71871079]
1672.58110765
Alpha: 0.000100
[ 112.68899732  -84.01119938  -24.68263174  -21.00088873  -21.71668162]
1672.58110765
Alpha: 0.001000
[ 112.68881437  -84.01104228  -24.66232204  -20.98060316  -21.69640993]
1672.58110774
Alpha: 0.010000
[ 112.68698753  -84.00947323  -24.46121539  -20.77973778  -21.49568404]
1672.58111645
Alpha: 0.100000
[ 112.66896732  -83.99396383  -22.63109556  -18.95202277  -19.6694

### Or we can use grid search to make this faster

In [21]:
from sklearn import grid_search

alphas = np.logspace(-10, 10, 21)
gs = grid_search.GridSearchCV(
    estimator  = linear_model.Ridge(),
    param_grid = {"alpha": alphas},
    scoring    = "mean_squared_error")

gs.fit(modeldata, y)

/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-10,   1.00000e-09,   1.00000e-08,   1.00000e-07,
         1.00000e-06,   1.00000e-05,   1.00000e-04,   1.00000e-03,
         1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)

### Best score

In [22]:
print(gs.best_score_)

-1814.09369133


#### mean squared error here comes in negative, so let's make it positive

In [23]:
print(-gs.best_score_)

1814.09369133


#### explains which grid_search setup worked best

In [24]:
print(gs.best_estimator_)

Ridge(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


#### shows all the grid pairings and their performances

In [25]:
print(gs.grid_scores_)

[mean: -1817.58711, std: 542.14315, params: {'alpha': 1e-10}, mean: -1817.58711, std: 542.14315, params: {'alpha': 1.0000000000000001e-09}, mean: -1817.58711, std: 542.14315, params: {'alpha': 1e-08}, mean: -1817.58711, std: 542.14315, params: {'alpha': 9.9999999999999995e-08}, mean: -1817.58711, std: 542.14315, params: {'alpha': 9.9999999999999995e-07}, mean: -1817.58711, std: 542.14317, params: {'alpha': 1.0000000000000001e-05}, mean: -1817.58707, std: 542.14331, params: {'alpha': 0.0001}, mean: -1817.58663, std: 542.14477, params: {'alpha': 0.001}, mean: -1817.58230, std: 542.15933, params: {'alpha': 0.01}, mean: -1817.54318, std: 542.30102, params: {'alpha': 0.10000000000000001}, mean: -1817.20111, std: 543.63587, params: {'alpha': 1.0}, mean: -1814.09369, std: 556.35563, params: {'alpha': 10.0}, mean: -1818.51694, std: 653.68607, params: {'alpha': 100.0}, mean: -2125.58777, std: 872.45270, params: {'alpha': 1000.0}, mean: -2458.08836, std: 951.30428, params: {'alpha': 10000.0}, me

## Gradient Descent

In [26]:
num_to_approach, start, steps, optimized = 6.2, 0.0, [-1, 1], False
while not optimized:
    current_distance = num_to_approach - start
    got_better = False
    next_steps = [start + i for i in steps]
    for n in next_steps:
        distance = np.abs(num_to_approach - n)
        if distance < current_distance:
            got_better = True
            print distance, "is better than", current_distance
            current_distance = distance
            start = n
    if got_better:
        print "found better solution! using", current_distance
        a += 1
    else:
        optimized = True
        print start, "is closest to", num_to_approach

5.2 is better than 6.2
found better solution! using 5.2
4.2 is better than 5.2
found better solution! using 4.2
3.2 is better than 4.2
found better solution! using 3.2
2.2 is better than 3.2
found better solution! using 2.2
1.2 is better than 2.2
found better solution! using 1.2
0.2 is better than 1.2
found better solution! using 0.2
6.0 is closest to 6.2


### Bonus:
Implement a stopping point, similar to what `n_iter` would do in gradient descent when we've reached "good enough".

In [ ]:
# <Code Here>

## Demonstration: Application of Gradient Descent 

In [27]:
lm = linear_model.SGDRegressor()
lm.fit(modeldata, y)
print("Gradient Descent R2 : %f" % lm.score(modeldata, y))
print("Gradient Descent MSE: %f" % metrics.mean_squared_error(y, lm.predict(modeldata)))

Gradient Descent R2 : 0.308238
Gradient Descent MSE: 1681.566641


### Check: Untuned, how well did gradient descent perform compared to OLS?

    Previous Result (from above):
    ~~~~ SUMMARY OF CROSS VALIDATION ~~~~
    Mean of MSE for all folds: 1673.089987
    Mean of R2 for all folds : 0.311911

Answer: ?

# Independent Practice: Bike data revisited

There are tons of ways to approach a regression problem. The regularization techniques appended to ordinary least squares optimizes the size of coefficients to best account for error. Gradient Descent also introduces learning rate (how aggressively do we solve the problem), epsilon (at what point do we say the error margin is acceptable), and iterations (when should we stop no matter what?)

For this deliverable, our goals are to:
- implement the gradient descent approach to our bike-share modeling problem
- show how gradient descent solves and optimizes the solution
- demonstrate the grid_search module

While exploring the Gradient Descent regressor object, you will build a grid search using the stochastic gradient descent estimator for the bike-share data set. Continue with either the model you evaluated last class or the simpler one from today. In particular, be sure to implement the "param_grid" in the grid search to get answers for the following questions:
- With a set of alpha values between $10^{-10}$ and $10^{-1}$, how does the mean squared error change?
- Based on the data, we know when to properly use L1 vs L2 regularization. By using a grid search with L1_ratios between 0 and 1 (increasing every 0.05), does that statement hold true? If not, did gradient descent have enough iterations?
- How do these results change when you alter the learning rate (eta0)?

**Bonus**: Can you see the advantages and disadvantages of using gradient descent after finishing this exercise?

### Starter Code

In [28]:
params = {} # put your gradient descent parameters here
gs = grid_search.GridSearchCV(
    estimator  = linear_model.SGDRegressor(),
    cv         = cross_validation.KFold(len(modeldata), n_folds = 5, shuffle = True),
    param_grid = params,
    scoring    = "mean_squared_error")

gs.fit(modeldata, y)

print("BEST ESTIMATOR")
print(-gs.best_score_)
print(gs.best_estimator_)
print("ALL ESTIMATORS")
print(gs.grid_scores_)

/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/mactzu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/hom

BEST ESTIMATOR
1688.69584914
SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)
ALL ESTIMATORS
[mean: -1688.69585, std: 96.60055, params: {}]


### Independent Practice Solution
This code shows the variety of challenges and some student gotchas. The plots will help showcase what should be learned.

1. With a set of alpha values between $10^{-10}$ and $10^{-1}$, how does the mean squared error change?
2. We know when to properly use L1 vs L2 regularization based on the data. By using a grid search with L1_ratios between 0 and 1 (increasing every 0.05), does that statement hold true?
    - (if it did not look like it, did gradient descent have enough iterations?)
3. How do results change when you alter the learning rate (power_t)?

In [ ]:
# <Code Here>

With the alphas available, it looks like at mean squared error stays generally flat with incredibly small alpha values, but starting at $10^{-3}$, the error begins to elbow. We probably do not have much of a different in performance with other alpha values.

In [ ]:
# <Code Here>

At alpha values of either 0.1 or 1, the L1_ratio works best closer to 1! Interesting.

At other values of alpha they should see similar results, though the graphs are not as clear.

In [ ]:
# <Code Here>

In [ ]:
# <Code Here>

In [ ]:
# <Code Here>

In [ ]:
# <Code Here>

Here it should be apparent that as the initial learning rate increases, the error should **also** increase. And what happens when the initial learning rate is too high? A dramatic increase in error. Students should recognize the importance of learning rate and what values it should be set at, the smaller generally the better.

In [ ]:
# <Code Here>